In [65]:
import sys, os, multiprocessing, urllib, csv
from PIL import Image
from io import BytesIO


def ParseData(data_file):
    csvfile = open(data_file, 'r')
    csvreader = csv.reader(csvfile)
    key_url_list = [line[:2] for line in csvreader][1:]
    return key_url_list


def DownloadImage(key_url):
    out_dir = os.path.join(os.getcwd(), 'output')

    (key, url) = key_url
    filename = os.path.join(out_dir, '%s.jpg' % key)

    if os.path.exists(filename):
        print('Image %s already exists. %%SVGkipping download.' % filename)
        return

    try:
        response = urllib.request.urlopen(url)
        image_data = response.read()
    except:
        print('Warning: Could not download image %s from %s' % (key, url))
        return

    try:
        pil_image = Image.open(BytesIO(image_data))
        pil_image = pil_image.resize((1200, 1200))
    except:
        print('Warning: Failed to parse image %s' % key)
        return


    try:
        pil_image.save(filename, format='JPEG', quality=90)
    except:
        print('Warning: Failed to save image %s' % filename)
        return


def Run():
    data_file = os.path.join(os.getcwd(), os.path.join('landmark-recognition-challenge', 'train.csv'))
    out_dir = os.path.join(os.getcwd(), 'output')

    if not os.path.exists(out_dir):
        os.mkdir(out_dir)

    key_url_list = ParseData(data_file)[:5200]
    pool = multiprocessing.Pool(processes=4)
    pool.map(DownloadImage, key_url_list)


if __name__ == '__main__':
    Run()


Image /Users/jonathanreich/Documents/recog-google/output/cacf8152e2d2ae60.jpg already exists. %SVGkipping download.
Image /Users/jonathanreich/Documents/recog-google/output/0a58358a2afd3e4e.jpg already exists. %SVGkipping download.
Image /Users/jonathanreich/Documents/recog-google/output/6b2bb500b6a38aa0.jpg already exists. %SVGkipping download.
Image /Users/jonathanreich/Documents/recog-google/output/b399f09dee9c3c67.jpg already exists. %SVGkipping download.
Image /Users/jonathanreich/Documents/recog-google/output/19ace29d77a5be66.jpg already exists. %SVGkipping download.
Image /Users/jonathanreich/Documents/recog-google/output/2c9c54b62f0a6a37.jpg already exists. %SVGkipping download.
Image /Users/jonathanreich/Documents/recog-google/output/0aac70a1de44ced6.jpg already exists. %SVGkipping download.
Image /Users/jonathanreich/Documents/recog-google/output/de770bc720f8e714.jpg already exists. %SVGkipping download.
Image /Users/jonathanreich/Documents/recog-google/output/dc9457d703e612a

In [14]:
import pandas as pd
import numpy as np
import os

In [15]:
a = pd.read_csv('landmark-recognition-challenge/train.csv')
OUTPUT_DIR  = 'output'


In [16]:
import keras

In [17]:
import tensorflow as tf

In [18]:
import glob
import numpy as np
t = glob.glob('output/' + '*.jpg')
from skimage import io
t[0]

'output/81fbd78fb1459162.jpg'

In [19]:
from skimage import io
import os
arr_list = []
for file in os.listdir('output/'):
    try:
        img = io.imread('output/' + file)
        input_arr = np.array([img])  # Convert single image to a batch.
        inpur_arr = np.expand_dims(input_arr, axis=0)  # or axis=1
        arr_list.append(input_arr)
    except:
        print('ll')


ll
ll


In [20]:
id_list = []
for file in os.listdir('output'):
    file_name, ext = os.path.splitext(file)
    id_list.append(file_name)

In [21]:
train_data = a[a.id.isin(id_list)]

In [22]:
from sklearn.model_selection import train_test_split

In [26]:
import glob
test_path = 'output/'
test_image_files = glob.glob(test_path + '*.jpg')
test_image_ids = [image_file.replace(
    '.jpg', '').replace(test_path, '') for image_file in test_image_files]


In [27]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_encoder = LabelEncoder()
train_info = a[a.id.isin(test_image_ids)]
labels = label_encoder.fit_transform(train_info['landmark_id'].values)
one_hot_encoder = OneHotEncoder(sparse=True, categories=train_data['landmark_id'].nunique())
train_info['labels'] = label_encoder.fit_transform(train_info['landmark_id'].values)
e = train_info['labels'].values.reshape(-1,1)
onehot = pd.get_dummies(train_info['labels'])

/Users/jonathanreich/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:
Y = onehot
Y = Y[:-1]
X = arr_list

In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2


In [33]:
from keras import Sequential, optimizers
from keras.callbacks import ModelCheckpoint
from keras.layers import BatchNormalization, Activation, Conv2D, Dense, Dropout, Flatten, GlobalMaxPool2D, MaxPooling2D, Softmax

In [34]:
input_shape = X_train[0].shape[1], X_train[0].shape[2], X_train[0].shape[3] 

In [35]:
modelBN = Sequential()
modelBN.add(Conv2D(filters=16, kernel_size=4, padding='same', use_bias=True, input_shape=input_shape))
modelBN.add(Activation("relu"))
modelBN.add(BatchNormalization())
modelBN.add(MaxPooling2D(pool_size=2))

modelBN.add(Conv2D(filters=32, kernel_size=3, padding='same', use_bias=True))
modelBN.add(Activation("relu"))
modelBN.add(BatchNormalization())
modelBN.add(MaxPooling2D(pool_size=2))

modelBN.add(Conv2D(filters=64, kernel_size=3, padding='same', use_bias=True))
modelBN.add(Activation("relu"))
modelBN.add(BatchNormalization())
modelBN.add(MaxPooling2D(pool_size=2))

modelBN.add(Conv2D(filters=128, kernel_size=2, padding='same', use_bias=True))
modelBN.add(Activation("relu"))
modelBN.add(BatchNormalization())
modelBN.add(MaxPooling2D(pool_size=2))

modelBN.add(Flatten())
modelBN.add(Dense(512, activation='relu'))
modelBN.add(Dropout(0.3))
modelBN.add(Dense(train_data['landmark_id'].nunique(), activation='softmax'))

modelBN.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 1200, 1200, 16)    784       
_________________________________________________________________
activation (Activation)      (None, 1200, 1200, 16)    0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1200, 1200, 16)    64        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 600, 600, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 600, 600, 32)      4640      
_________________________________________________________________
activation_1 (Activation)    (None, 600, 600, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 600, 600, 32)      1

In [36]:
adam = optimizers.Adam(lr=0.02)
modelBN.compile(optimizer='adamax', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_BatchNorm.hdf5', 
                               verbose=1, save_best_only=True)

epochs_batch = 15
X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_tet).astype('float32')

hist_BN = modelBN.fit(X_train, Y_train, validation_data=(X_test, Y_test), 
epochs=epochs_batch, batch_size=64, callbacks=[checkpointer], verbose=1)



In [ ]:
modelBN.load_weights('saved_models/weights.best.from_BatchNorm.hdf5')

landmark_pred = [np.argmax(modelBN.predict(np.expand_dims(tensor, axis=0))) for tensor in X_val]
test_accuracy = np.sum(np.array(landmark_pred) == np.argmax(Y_val, axis=1)) / len(landmark_pred)
print('Test accuracy:', test_accuracy)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rotation_range=45, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    zoom_range=0.3)

valid_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(X_train, Y_train, batch_size= 128)
valid_generator = valid_datagen.flow(X_val, Y_val, batch_size= 128)
epochs_aug = 50

hist_aug = model.fit_generator(train_generator, steps_per_epoch=6480//128, epochs=epochs_aug,
                    validation_data=valid_generator, validation_steps=805//128,
                    callbacks=[checkpointer], verbose=1)


In [ ]:
# import albumentations
# transforms_672 = albumentations.Compose([
#     albumentations.Resize(672, 672),
#     albumentations.Normalize()
# ])

# transforms_768 = albumentations.Compose([
#     albumentations.Resize(768, 768),
#     albumentations.Normalize()
# ])
# transforms_512 = albumentations.Compose([
#     albumentations.Resize(512, 512),
#     albumentations.Normalize()
# ])

# import cv2
# for row in os.listdir('outputs'):
#     image = cv2.imread(row)
#     image = image[:, :, ::-1]

#     res0 = self.transform672(image=image)
#     image0 = res0['image'].astype(np.float32)
#     image0 = image0.transpose(2, 0, 1)        

#     res1 = self.transform768(image=image)
#     image1 = res1['image'].astype(np.float32)
#     image1 = image1.transpose(2, 0, 1)    

#     res3 = self.transform512(image=image)
#     image3 = res3['image'].astype(np.float32)        
#     image3 = image3.transpose(2, 0, 1)   
